In [19]:
import os
import time

In [ ]:
os.environ['KERAS_BACKEND']='tensorflow'

In [1]:
import numpy as np
import keras
#import keras_nlp
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


2024-07-06 19:51:13.784117: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-06 19:51:13.802217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-06 19:51:13.827488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-06 19:51:13.827530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-06 19:51:13.843837: I tensorflow/core/platform/cpu_feature_gua

In [2]:
#keras.mixed_precision.set_global_policy("mixed_float16")
print(tf.__version__)
#tf.compat.v1.disable_eager_execution()

2.16.2


In [3]:
import s3fs


In [4]:
root = 's3://differential-privacy-datasets'
wikitext2_root = root + '/kaggle-wikitext/wikitext-2/'
train_file = wikitext2_root + 'wiki.train.tokens'
test_file  = wikitext2_root + 'wiki.test.tokens'
valid_file = wikitext2_root + 'wiki.valid.tokens'
unittest_file = wikitext2_root + 'unittest.tokens'

In [5]:
BATCH_SIZE = 4
NUM_EPOCHS = 1
SEQUENCE_LENGTH = 512
SHUFFLE_SIZE = 128
#BLOCK_SIZE = 128

In [6]:
# Encode input text using the tokenizer provided
def encode(text, tokenizer, max_length=SEQUENCE_LENGTH):
    tokens = tokenizer.encode(text, padding='max_length', truncation=True, max_length=max_length)
    input_ids = tf.constant(tokens, dtype=tf.int32)
    attention_mask = tf.constant([1 if token != tokenizer.pad_token_id else 0 for token in tokens], dtype=tf.int32)
    return input_ids, attention_mask

In [7]:
def load_dataset(file_path, tokenizer, max_length=SEQUENCE_LENGTH):
    def generator_fn():
        fs = s3fs.S3FileSystem(anon=True)
        with fs.open(file_path, 'r', encoding='utf-8') as fd:
            for line in fd:
                sline = line.strip()
                if len(sline) > 0:
                    yield encode(sline, tokenizer, max_length=max_length)
                
    dataset = tf.data.Dataset.from_generator(
        generator_fn, 
        output_signature=
            (tf.TensorSpec(shape=(None,), dtype=tf.int32),
             tf.TensorSpec(shape=(None,), dtype=tf.int32))
    )
    return dataset.shuffle(SHUFFLE_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([max_length], [max_length]))

In [8]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
#gpt2_tokenizer.padding_side = 'left'


In [9]:
train_dataset = load_dataset(train_file, gpt2_tokenizer).shuffle(buffer_size=SHUFFLE_SIZE)
test_dataset  = load_dataset(test_file, gpt2_tokenizer)
valid_dataset = load_dataset(valid_file, gpt2_tokenizer)
unittest_dataset = load_dataset(unittest_file, gpt2_tokenizer)

2024-07-06 19:52:32.597936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-06 19:52:32.646953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-06 19:52:32.648761: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
for ele in unittest_dataset:
    print(ele)

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[15496,  2159,    13, ..., 50256, 50256, 50256],
       [34784,  1365,  1110, ..., 50256, 50256, 50256],
       [49488,   318,  2901, ..., 50256, 50256, 50256]], dtype=int32)>, <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>)


2024-07-06 19:52:33.641092: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
print(unittest_dataset)

<_PaddedBatchDataset element_spec=(TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), TensorSpec(shape=(None, 512), dtype=tf.int32, name=None))>


In [12]:
model_name = 'gpt2'
gpt2_lm = TFGPT2LMHeadModel.from_pretrained(model_name)
gpt2_lm.resize_token_embeddings(len(gpt2_tokenizer)) 

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [13]:
gpt2_lm.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124439808 
 er)                                                             
                                                                 
Total params: 124439808 (474.70 MB)
Trainable params: 124439808 (474.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
'''
optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=1.1,
    num_microbatches=8,
    learning_rate=1e-4
)
'''
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)
'''
optimizer + tf.keras.optimizers.SGD(learning_rate = 0.01,
                                momentum=0.0, 
                                nesterov=False, 
                                name='SGD')
'''                                
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
gpt2_lm.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [16]:
def generate(input_text, max_length=256):
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors='tf')
    attention_mask = tf.ones_like(input_ids)
    outputs   = gpt2_lm.generate(input_ids, max_length = max_length, attention_mask=attention_mask, 
                                 num_return_sequences=5,
                                 no_repeat_ngram_size=2,  # Avoid repeating n-grams of size 2
                                 top_k=25,  # Limits the sampling pool to top_k tokens
                                 do_sample=True,
                                 temperature = 0.9,
                                 #top_p=0.95, 
                                 pad_token_id=gpt2_tokenizer.eos_token_id)
    gen_text  = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return gen_text

In [17]:
print(generate('I went on a trip to see Tajmahal in Agra. My trip was'))

I went on a trip to see Tajmahal in Agra. My trip was over when we got to the airport. We arrived at about 6AM with my girlfriend and our dog and she had taken a bag from the plane. I was about to go see the Taj Mahal when her car was pulled over on the highway. Luckily, my friend was able to pick me up and we had to leave. After that, the next night we were gone! When we left, I found out who we are. She told me I've been in Taj Mohalas for 3 years and the one thing she told her was that it wasn't that bad and that they never told us where we went. It was great! The tour was so good, we could have done it any day. The people I met on my travels are amazing.

I am a big fan of the Mahaan, which means "mahaana". It is a Hindu holiday, meaning that all the people who live there must be Hindu. This is great for the children and for those who do not have a lot of relatives or a family member like me. However, when you come to Indian holidays such as Muharram, Mardi Gras and Mahat


In [20]:
st = time.time()
history = gpt2_lm.fit(
    train_dataset,
    epochs=NUM_EPOCHS,
    validation_data=valid_dataset
)
en = time.time()

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
   5942/Unknown - 4012s 668ms/step - loss: 0.0040 - accuracy: 0.9995

2024-07-06 21:03:22.568375: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-06 21:03:22.568435: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 12903150190782255270
2024-07-06 21:03:22.568445: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 7218944536086067346
2024-07-06 21:03:22.568454: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 13044163714173017470
2024-07-06 21:03:22.568463: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 103420089854845379
2024-07-06 21:03:22.568471: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 644235837523252059
2024-07-06 21:03:22.568479: I tensorflow/core/framework/local_rendezvou

5942/5942 [==============================] - 4188s 697ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 1.3683e-07 - val_accuracy: 1.0000


In [ ]:
loss, accuracy = gpt2_lm.evaluate(test_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [21]:
print(f'Training time {(en-st)/3600} hours')

Training time 1.1632336160209444 hours


In [22]:
print(generate('I went on a trip to see Tajmahal in Agra. My trip was'))

I went on a trip to see Tajmahal in Agra. My trip was"" and"(""!"")".""!"" ("The":""T"""""-""("S"M"B")"",""s"L"I"R"D"E"A"U"H"]""O"Y"F"",""'"J";""W"!" ""P""["i"";"'"G"["He"
"?""1">";"C" &":"'""#"/"" with" to">""*"&" from" of" a" by"":" =","2""..." for" in" the"-"in" which" on" as" it" -"<"K""]"+"Z"And"...""by" one""."!!"" :".".""?"!" #" /"

"What" is".."!!!"@"In" i"a" 2"...."..."..."!!!!"!?"" "It"?!""!!"
